## Repository
# https://github.com/TI-Russia/datafest2019

In [1]:
import requests
import pandas as pd
import json
import re
from bs4 import BeautifulSoup as bs

In [ ]:
def get_json(url):

    r = requests.get(url)
    data = json.loads(r.text)

    return data


def getstate(x, regions_matches):
    match = re.search(r'(?:ქ\.|\s)*('+'|'.join(regions_matches.keys())+')\s*,*', x)

    if match:

        return regions_matches[match.group(1).strip()]

## Regions names
# http://shorturl.at/foBS8

## Regions matches
# http://shorturl.at/yGIN0

    regions_names_url = 'http://shorturl.at/foBS8'
    regions_matches_url = 'http://shorturl.at/yGIN0'

    regions_names = get_json(regions_names_url)
    regions_matches = get_json(regions_matches_url)

    r = requests.get('https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Georgia_(country)')
    municipality = pd.read_html(r.text)[0]

    municipality = municipality[['Name in Georgian','Administrative Region']]
    municipality.set_index('Name in Georgian', inplace=True)
    json.loads(municipality.to_json(force_ascii=False, orient='columns'))['Administrative Region']

## Data
# http://shorturl.at/fAZ69

    data = json.loads(open('/Users/schtsch/Desktop/Georgian_Declarations/2018/2018_dcl_done_data.json').read())

    DF = pd.DataFrame()

    for d in data:
        p = d.get('purchase', None)
        if p:
            df = pd.DataFrame(p)
            DF = DF.append(df)

    DF.loc[:,'region'] = DF.location_area.apply(getstate, args=(regions_matches,))

    df = DF.groupby('region').size()

    count_data = pd.DataFrame(df)
    count_data.loc[:,'name_en'] = count_data.index

    done_table = pd.merge(count_data, pd.DataFrame(regions_names), on='name_en')

    done_table.to_csv('/Users/schtsch/Desktop/Georgian_Declarations/2018/', index=False)